In [1]:
import pandas as pd
import mdtraj as md
import numpy as np
from numpy.random import seed
from numpy.random import shuffle
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from __future__ import print_function
import seaborn as sns
from matplotlib.ticker import NullFormatter, MaxNLocator
from pandas.plotting import scatter_matrix
import matplotlib.ticker as ticker
#import plotly.graph_objects as go
import scipy as sp
from itertools import chain, combinations
import matplotlib as mpl
from matplotlib.lines import Line2D
from scipy import spatial
from scipy.spatial import ConvexHull
from scipy.optimize import curve_fit
import scipy.stats as stats
#import statsmodels.stats.weightstats
from matplotlib import path
import matplotlib
from scipy.stats import probplot,shapiro, sem
#import statsmodels.api as sm
from scipy.interpolate import make_interp_spline
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from scipy import stats
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,r2_score, mean_squared_error
from sklearn.metrics import RocCurveDisplay, roc_auc_score
from sklearn.linear_model import RidgeCV
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay,classification_report
from sklearn.svm import SVC, SVR
from sklearn.preprocessing import LabelEncoder, label_binarize, StandardScaler

from matplotlib import cm
from collections import Counter
from numpy import linspace
#import umap.umap_ as umap
#import pylab
import os
#import PIL
from scipy.ndimage import gaussian_filter, uniform_filter1d

In [2]:
# import soursop
import afrc
import soursop
import MDAnalysis
import pandas as pd
import numpy as np
from soursop.sstrajectory import SSTrajectory
from Bio.PDB import *
from itertools import chain
import h5py

In [3]:
print("MAKE SURE afrc_venv_ virtual environment is enabled with conda")
print("MAKE SURE python version is 3.9.18")
import sys
sys.version

MAKE SURE afrc_venv_ virtual environment is enabled with conda
MAKE SURE python version is 3.9.18


'3.9.18 (main, Jan 26 2024, 16:04:25) \n[GCC 8.5.0 20210514 (Red Hat 8.5.0-20)]'

# Some Functions

In [4]:
def RSA_based_fC(protein_var,protein_name,poly_id,
                       GW_moment_var,every_ith_snap,GW_every_ith_snap,radius_):
    x_total=[]
    y_total=[]
    if poly_id=='protein':
        protein_label=protein_name
        temp_protein=protein_var.copy()
        x_total.append(temp_protein.RSA.values)
        y_total.append(temp_protein.ratio.values)
        x_total=list(chain.from_iterable(x_total))
        y_total=list(chain.from_iterable(y_total))
        poly_var=protein_var.copy()
        protein_pro=poly_var[['ratio','RSA']].iloc[:every_ith_snap,:].copy()
        protein_pro['polymer_id']=np.repeat(protein_label,protein_pro.shape[0])
        del x_total, y_total, temp_protein
    else:
        return print('ERROR')

    GW_po=GW_moment_var[['ratio','RSA']][::GW_every_ith_snap].copy()
    GW_po['polymer_id']=np.repeat('GW',GW_po.shape[0])
    
    #calculate mean and stdev values (must keep same mean and stdev values)
    upto_snapshots=1000000
    GW_mean_ratio=np.mean(GW_po['ratio'].values[0:(upto_snapshots+1)])
    GW_std_ratio=np.std(GW_po['ratio'].values[0:(upto_snapshots+1)])
    GW_mean_RSA=np.mean(GW_po['RSA'].values[0:(upto_snapshots+1)])
    GW_std_RSA=np.std(GW_po['RSA'].values[0:(upto_snapshots+1)])
        
    combined_pro_po=pd.concat([GW_po,protein_pro],axis=0,ignore_index=True)
    combined_pro_po['stdd_ratio']=(combined_pro_po['ratio'].values-GW_mean_ratio)/(GW_std_ratio)
    combined_pro_po['stdd_RSA']=(combined_pro_po['RSA'].values-GW_mean_RSA)/(GW_std_RSA)
    po_x=combined_pro_po[combined_pro_po.polymer_id=='GW']['stdd_RSA'].values
    po_y=combined_pro_po[combined_pro_po.polymer_id=='GW']['stdd_ratio'].values
    pro_x=combined_pro_po[combined_pro_po.polymer_id==protein_label]['stdd_RSA'].values
    pro_y=combined_pro_po[combined_pro_po.polymer_id==protein_label]['stdd_ratio'].values
    
    GW_points=np.c_[po_x, po_y]
    protein_points=np.c_[pro_x, pro_y]    
    tree_GW=spatial.cKDTree(GW_points)
    tree_protein=spatial.cKDTree(protein_points)

    GW_not_in_range=[]
    j=0
    for point in GW_points:

        if not tree_protein.query_ball_point(point,radius_):
            GW_not_in_range.append(point)
        j+=1
        if j%100000==0:
            print(f'{j} GW snapshots completed')
                
    fC_by_distance=(GW_points.shape[0]-len(GW_not_in_range))/(GW_points.shape[0])
    return fC_by_distance    


In [5]:
def RSA_Rs_protein_3dplot_against_GW(protein_var,protein_label,second_obj,provided_color):
    
    x_variable= 'RSA'
    y_variable=r'$R_s$'
    global fC_value, grid_protein, grid_GW

    testeq_GW= pd.read_csv('../segmented_chain_project/reference_GW_chainlen_100_for_RSA.csv')
    
    plt.rcParams["font.weight"] = "regular"
    plt.rcParams["axes.labelweight"] = "regular"
    
    
    x_polmodel_GW=[]
    y_polmodel_GW=[]
    x_total=[]
    y_total=[]

    scatter_markers=['d','x',4,'o','v','^','1','8','s','p','P','*','X','D',9]


    x_polmodel_GW.append(testeq_GW['RSA'].values)
    y_polmodel_GW.append(testeq_GW['ratio'].values)

    if second_obj=='protein':        
        temp_protein =  protein_var.copy()

        x_total.append(temp_protein.RSA.values)
        y_total.append(temp_protein.ratio.values)

        fC_value=RSA_based_fC(temp_protein,
                       protein_label,
                       second_obj,testeq_GW,
                       temp_protein.shape[0],
                       1,0.1)            
        del temp_protein
    else:
        print('ERROR. FIX ERROR')

    x_total=list(chain.from_iterable(x_total))
    y_total=list(chain.from_iterable(y_total))
    x_polmodel_GW=list(chain.from_iterable(x_polmodel_GW))
    y_polmodel_GW=list(chain.from_iterable(y_polmodel_GW))

    # Set up default x and y limits
    max_x_val=1
    max_y_val=30
    xlims = [0,max_x_val]
    ylims = [0,max_y_val]
    if testeq_GW['RSA'].max()>max_x_val or testeq_GW['RSA'].min()<0:
        return print('ERROR')
    elif max(x_total)>max_x_val or min(x_total)<0:
        return print('ERROR')        
    elif testeq_GW['ratio'].max()>max_y_val or testeq_GW['ratio'].min()<0:
        return print('ERROR')
    elif max(y_total)>max_y_val or min(y_total)<0:
        return print('ERROR')        

    # Set up your x and y labels
    xlabel = x_variable
    ylabel = y_variable

    # Define the locations for the axes

    left, width = 0.12, 0.55
    bottom, height = 0.12, 0.55
    bottom_h = left_h = left+width
    rect_temperature = [left, bottom, width, height] # dimensions of temp plot
    rect_histx = [left, bottom_h, width, 0.25] # dimensions of x-histogram
    rect_histy = [left_h, bottom, 0.25, height] # dimensions of y-histogram


    # Set up the size of the figure
    fig = plt.figure(1, figsize=(5.2,5))

    # Make the three plots
    axTemperature = plt.axes(rect_temperature) # temperature plot
    axHistx = plt.axes(rect_histx) # x histogram
    axHisty = plt.axes(rect_histy) # y histogram


    axHistx.axes.get_xaxis().set_visible(False)
    axHisty.axes.get_yaxis().set_visible(False)


    # Find the min/max of the data
    xmin = min(xlims)
    xmax = max(xlims)
    ymin = min(ylims)
    ymax = max(ylims)

    # Remove the inner axes numbers of the histograms
    nullfmt = NullFormatter()
    #
    # Define the number of bins
    nxbins = 100
    nybins = 50
    nbins = nxbins+nybins

    xbins = linspace(start = xmin, stop = xmax, num = nxbins)
    ybins = linspace(start = ymin, stop = ymax, num = nybins)
    xcenter = (xbins[0:-1]+xbins[1:])/2.0
    ycenter = (ybins[0:-1]+ybins[1:])/2.0
    aspectratio = (1.0*(xmax - 0))/(1.38*ymax - 0)


    X = xcenter
    Y = ycenter

    axTemperature.scatter(x_polmodel_GW,y_polmodel_GW,
                      marker=scatter_markers[3],s=1,alpha=0.6,color='black',
                       label='GW')
    
    axTemperature.scatter(x_total,
               y_total,
               marker=scatter_markers[0],s=1,alpha=0.6,color=provided_color,
                                      label=protein_label)                



    #Plot the axes labels
    axTemperature.set_xlabel(xlabel,fontsize=12,labelpad = 1)
    axTemperature.set_ylabel(ylabel,fontsize=12, labelpad = 1)

    #Set up the plot limits
    axTemperature.set_xlim(xlims)
    axTemperature.set_ylim(ylims)

    #Set up the histogram bins
    xbins = np.arange(xmin, xmax, (xmax-xmin)/nbins)
    ybins = np.arange(ymin, ymax, (ymax-ymin)/nbins)


    axHistx.hist(x_polmodel_GW, bins=xbins, color = 'black',histtype='step',
                linewidth = 0.7,density=True)
    axHistx.hist(x_total, bins=xbins, color = provided_color,histtype='step',
                label=protein_label,linewidth = 0.7,density=True)

    axHisty.hist(y_polmodel_GW, bins=np.arange(min(y_total),max(y_total),1), color = 'black',orientation='horizontal',histtype='step',
                linewidth = 0.7,density=True)

    axHisty.hist(y_total, bins=ybins, color = provided_color,orientation='horizontal',histtype='step',
                label=protein_label,linewidth = 0.7,density=True)    
    
    #original_n_divisions=100
    step_x=0.02#(min(x_polmodel_GW)-max(x_polmodel_GW))/original_n_divisions
    step_y=0.6#(min(y_polmodel_GW)-max(y_polmodel_GW))/original_n_divisions
    max_value_x=max(xlims)
    n_divisions_x=round((max_value_x-min(xlims))/step_x)
    
    max_value_y=max(ylims)
    n_divisions_y=round((max_value_y-min(ylims))/step_y)
    #gridx = np.linspace(min(x_polmodel_GW), max(x_polmodel_GW), n_divisions_x)
    gridx = np.linspace(min(xlims), max(xlims), n_divisions_x)
    #gridy = np.linspace(min(y_polmodel_GW), max(y_polmodel_GW), n_divisions_y)
    gridy = np.linspace(min(ylims), max(ylims), n_divisions_y)
    
    grid_protein, _, _ = np.histogram2d(x_total, y_total, bins=[gridx, gridy])
    grid_GW, _, _ = np.histogram2d(x_polmodel_GW, y_polmodel_GW, bins=[gridx, gridy])

    axTemperature.text(0.7,0.8,
                       '$f_C$= '+format(fC_value,'0.3f'),
                      transform=axTemperature.transAxes,fontsize=8)
    axTemperature.text(0.7,0.74,
                       '$f_A$= '+format(np.count_nonzero(grid_protein)/np.count_nonzero(grid_GW),'0.3f'),
                      transform=axTemperature.transAxes,fontsize=8)    

    axTemp_legend=axTemperature.legend(fontsize=7,loc='upper right')

    frame = axTemp_legend.get_frame()
    frame.set_linewidth(1)
    frame.set_edgecolor('black')
    #axHisty.hist(y, bins=ybins, orientation='horizontal', color = provided_color)






    #Set up the histogram limits
    axHistx.set_xlim( xmin, xmax )
    axHisty.set_ylim( ymin, ymax )

    axHistx.set_ylabel('Prob. Density',fontsize=12)
    axHisty.set_xlabel('Prob. Density',fontsize=12)


    #Cool trick that changes the number of tickmarks for the histogram axes
    axHisty.xaxis.set_major_locator(MaxNLocator(4))
    axHistx.yaxis.set_major_locator(MaxNLocator(3))

    for i in ['top', 'left', 'right', 'bottom']:
        axHistx.spines[i].set_linewidth(1)
        axHisty.spines[i].set_linewidth(1)
        axTemperature.spines[i].set_linewidth(1)

    axHistx.minorticks_on()
    axHistx.tick_params(axis='y', which='major', labelsize=10, width = 0.8, pad = 2)
    axHistx.tick_params(axis='y', which='minor', width = 0.6)
    axHisty.minorticks_on()
    axHisty.tick_params(axis='x', which='major', labelsize=10, width = 0.8, pad = 2,labelrotation=270)
    axHisty.tick_params(axis='x', which='minor', width = 0.6)
    axTemperature.minorticks_on()
    axTemperature.tick_params(axis='both', which='major', labelsize=10, width = 0.8, pad = 2)
    axTemperature.tick_params(axis='both', which='minor', width = 0.6)    
    plt.setp(axTemperature.get_yticklabels()[-1], visible=False)
    plt.setp(axTemperature.get_xticklabels()[-1], visible=False)
    #fig.subplots_adjust(hspace=0.85,wspace=0.85)

    # Save to a File
    filename = 'GW_3d_plot'
    #plt.savefig(f'figures/GW_{protein_label}_3dplot.png', dpi = 1000, bbox_inches='tight')
    return 



In [6]:
def protein_3dplot_against_GW(protein_var,protein_label,second_obj,provided_color):
    
    x_variable= r'$R_g\left/R_g^{theta}\right.$' 
    y_variable=r'$R_s$'
    global grid_protein, grid_GW

    testeq_GW= pd.read_csv('../segmented_chain_project/reference_GW_chainlen_100_for_RSA.csv')
    
    plt.rcParams["font.weight"] = "regular"
    plt.rcParams["axes.labelweight"] = "regular"
    
    
    x_polmodel_GW=[]
    y_polmodel_GW=[]
    x_total=[]
    y_total=[]

    scatter_markers=['d','x',4,'o','v','^','1','8','s','p','P','*','X','D',9]

    GW_rg = testeq_GW['Rg2']**0.5
    GW_rg_mean = GW_rg.mean()
    GW_rg_by_rg_mean = GW_rg/GW_rg_mean
    testeq_GW['Rg/Rg_mean'] = GW_rg_by_rg_mean
    
    x_polmodel_GW.append(GW_rg_by_rg_mean)
    y_polmodel_GW.append(testeq_GW['ratio'].values)
    
    if second_obj=='protein':        
        temp_protein =  protein_var.copy()

        x_total.append(temp_protein['Rg/Rg_theta'].values)
        y_total.append(temp_protein.ratio.values)

        # fC_value=RSA_based_fC(temp_protein,
        #                protein_label,
        #                second_obj,testeq_GW,
        #                temp_protein.shape[0],
        #                1,0.1)            
        del temp_protein
    else:
        print('ERROR. FIX ERROR')

    x_total=list(chain.from_iterable(x_total))
    y_total=list(chain.from_iterable(y_total))
    x_polmodel_GW=list(chain.from_iterable(x_polmodel_GW))
    y_polmodel_GW=list(chain.from_iterable(y_polmodel_GW))

    # Set up default x and y limits
    max_x_val=3
    max_y_val=30
    xlims = [0,max_x_val]
    ylims = [0,max_y_val]
    if testeq_GW['Rg/Rg_mean'].max()>max_x_val or testeq_GW['Rg/Rg_mean'].min()<0:
        return print('ERROR')
    elif max(x_total)>max_x_val or min(x_total)<0:
        return print('ERROR')        
    elif testeq_GW['ratio'].max()>max_y_val or testeq_GW['ratio'].min()<0:
        return print('ERROR')
    elif max(y_total)>max_y_val or min(y_total)<0:
        return print('ERROR')        

    # Set up your x and y labels
    xlabel = x_variable
    ylabel = y_variable

    # Define the locations for the axes

    left, width = 0.12, 0.55
    bottom, height = 0.12, 0.55
    bottom_h = left_h = left+width
    rect_temperature = [left, bottom, width, height] # dimensions of temp plot
    rect_histx = [left, bottom_h, width, 0.25] # dimensions of x-histogram
    rect_histy = [left_h, bottom, 0.25, height] # dimensions of y-histogram


    # Set up the size of the figure
    fig = plt.figure(1, figsize=(5.2,5))

    # Make the three plots
    axTemperature = plt.axes(rect_temperature) # temperature plot
    axHistx = plt.axes(rect_histx) # x histogram
    axHisty = plt.axes(rect_histy) # y histogram


    axHistx.axes.get_xaxis().set_visible(False)
    axHisty.axes.get_yaxis().set_visible(False)


    # Find the min/max of the data
    xmin = min(xlims)
    xmax = max(xlims)
    ymin = min(ylims)
    ymax = max(ylims)

    # Remove the inner axes numbers of the histograms
    nullfmt = NullFormatter()
    #
    # Define the number of bins
    nxbins = 100
    nybins = 50
    nbins = nxbins+nybins

    xbins = linspace(start = xmin, stop = xmax, num = nxbins)
    ybins = linspace(start = ymin, stop = ymax, num = nybins)
    xcenter = (xbins[0:-1]+xbins[1:])/2.0
    ycenter = (ybins[0:-1]+ybins[1:])/2.0
    aspectratio = (1.0*(xmax - 0))/(1.38*ymax - 0)


    X = xcenter
    Y = ycenter

    axTemperature.scatter(x_polmodel_GW,y_polmodel_GW,
                      marker=scatter_markers[3],s=1,alpha=0.6,color='black',
                       label='GW')
    
    axTemperature.scatter(x_total,
               y_total,
               marker=scatter_markers[0],s=1,alpha=0.6,color=provided_color,
                                      label=protein_label)                



    #Plot the axes labels
    axTemperature.set_xlabel(xlabel,fontsize=12,labelpad = 1)
    axTemperature.set_ylabel(ylabel,fontsize=12, labelpad = 1)

    #Set up the plot limits
    axTemperature.set_xlim(xlims)
    axTemperature.set_ylim(ylims)

    #Set up the histogram bins
    xbins = np.arange(xmin, xmax, (xmax-xmin)/nbins)
    ybins = np.arange(ymin, ymax, (ymax-ymin)/nbins)


    axHistx.hist(x_polmodel_GW, bins=xbins, color = 'black',histtype='step',
                linewidth = 0.7,density=True)
    axHistx.hist(x_total, bins=xbins, color = provided_color,histtype='step',
                label=protein_label,linewidth = 0.7,density=True)

    axHisty.hist(y_polmodel_GW, bins=np.arange(min(y_total),max(y_total),1), color = 'black',orientation='horizontal',histtype='step',
                linewidth = 0.7,density=True)

    axHisty.hist(y_total, bins=ybins, color = provided_color,orientation='horizontal',histtype='step',
                label=protein_label,linewidth = 0.7,density=True)    
    
    #original_n_divisions=100
    step_x=0.02#(min(x_polmodel_GW)-max(x_polmodel_GW))/original_n_divisions
    step_y=0.6#(min(y_polmodel_GW)-max(y_polmodel_GW))/original_n_divisions
    max_value_x=max(xlims)
    n_divisions_x=round((max_value_x-min(xlims))/step_x)
    
    max_value_y=max(ylims)
    n_divisions_y=round((max_value_y-min(ylims))/step_y)
    #gridx = np.linspace(min(x_polmodel_GW), max(x_polmodel_GW), n_divisions_x)
    gridx = np.linspace(min(xlims), max(xlims), n_divisions_x)
    #gridy = np.linspace(min(y_polmodel_GW), max(y_polmodel_GW), n_divisions_y)
    gridy = np.linspace(min(ylims), max(ylims), n_divisions_y)
    
    grid_protein, _, _ = np.histogram2d(x_total, y_total, bins=[gridx, gridy])
    grid_GW, _, _ = np.histogram2d(x_polmodel_GW, y_polmodel_GW, bins=[gridx, gridy])

    # axTemperature.text(0.7,0.8,
    #                    '$f_C$= '+format(fC_value,'0.3f'),
    #                   transform=axTemperature.transAxes,fontsize=8)
    axTemperature.text(0.7,0.74,
                       '$f_A$= '+format(np.count_nonzero(grid_protein)/np.count_nonzero(grid_GW),'0.3f'),
                      transform=axTemperature.transAxes,fontsize=8)    

    axTemp_legend=axTemperature.legend(fontsize=7,loc='upper right')

    frame = axTemp_legend.get_frame()
    frame.set_linewidth(1)
    frame.set_edgecolor('black')
    #axHisty.hist(y, bins=ybins, orientation='horizontal', color = provided_color)






    #Set up the histogram limits
    axHistx.set_xlim( xmin, xmax )
    axHisty.set_ylim( ymin, ymax )

    axHistx.set_ylabel('Prob. Density',fontsize=12)
    axHisty.set_xlabel('Prob. Density',fontsize=12)


    #Cool trick that changes the number of tickmarks for the histogram axes
    axHisty.xaxis.set_major_locator(MaxNLocator(4))
    axHistx.yaxis.set_major_locator(MaxNLocator(3))

    for i in ['top', 'left', 'right', 'bottom']:
        axHistx.spines[i].set_linewidth(1)
        axHisty.spines[i].set_linewidth(1)
        axTemperature.spines[i].set_linewidth(1)

    axHistx.minorticks_on()
    axHistx.tick_params(axis='y', which='major', labelsize=10, width = 0.8, pad = 2)
    axHistx.tick_params(axis='y', which='minor', width = 0.6)
    axHisty.minorticks_on()
    axHisty.tick_params(axis='x', which='major', labelsize=10, width = 0.8, pad = 2,labelrotation=270)
    axHisty.tick_params(axis='x', which='minor', width = 0.6)
    axTemperature.minorticks_on()
    axTemperature.tick_params(axis='both', which='major', labelsize=10, width = 0.8, pad = 2)
    axTemperature.tick_params(axis='both', which='minor', width = 0.6)    
    plt.setp(axTemperature.get_yticklabels()[-1], visible=False)
    plt.setp(axTemperature.get_xticklabels()[-1], visible=False)
    #fig.subplots_adjust(hspace=0.85,wspace=0.85)

    # Save to a File
    filename = 'GW_3d_plot'
    #plt.savefig(f'figures/GW_{protein_label}_3dplot.png', dpi = 1000, bbox_inches='tight')
    return 



# generate a dataframe connecting directory with seq_name in their property df

In [7]:
seq_name_AFRC = pd.read_csv('../holehouse_project/IDRome_shape_mean_size_mean_added.csv')

In [8]:
seq_name_dir_df = pd.read_csv('../larsen_paper_2023_compaction/seq_name_dir_df.csv')
seq_name_dir_df

,seq_name,seq_dir,protein_uniprot_id
0,H0YL14_1_46,IDROME_larsen/IDRome_v4/H0/YL/14/1_46,H0YL14
1,H0YL09_1_131,IDROME_larsen/IDRome_v4/H0/YL/09/1_131,H0YL09
2,H0YKK7_1_89,IDROME_larsen/IDRome_v4/H0/YK/K7/1_89,H0YKK7
3,H0YKK7_135_164,IDROME_larsen/IDRome_v4/H0/YK/K7/135_164,H0YKK7
4,H0YKK7_430_550,IDROME_larsen/IDRome_v4/H0/YK/K7/430_550,H0YKK7
...,...,...,...
28053,A3QJZ6_111_153,IDROME_larsen/IDRome_v4/A3/QJ/Z6/111_153,A3QJZ6
28054,A3QJZ7_106_152,IDROME_larsen/IDRome_v4/A3/QJ/Z7/106_152,A3QJZ7
28055,A3KN83_1001_1044,IDROME_larsen/IDRome_v4/A3/KN/83/1001_1044,A3KN83
28056,A3KN83_1_250,IDROME_larsen/IDRome_v4/A3/KN/83/1_250,A3KN83


In [9]:
seq_name_AFRC

,seq_name,UniProt_ID,N,nu,SPR,ete2_Rg2,S,Delta,Rg/nm,Ree/nm,...,z_omega_+,z_omega_-,z_omega_h,fdomain,nu_svr,SPR_svr,AFRC_mean_rg_theta,rg_rg_theta_mean,mean_inst_ratio,mean_RSA
0,A0A024RBG1_145_181,A0A024RBG1,37,0.558,10.284649,6.304027,0.933961,0.225950,1.606561,3.843011,...,0.000000,0.000000,-0.531910,0.000000,0.565,10.288,15.191941,1.057621,5.808555,0.442952
1,A0A075B6T7_1_32,A0A075B6T7,32,0.525,10.092351,6.114494,0.901807,0.212670,1.448141,3.390487,...,-0.043142,0.000000,0.061436,0.000000,0.532,10.107,14.102458,1.026155,5.598546,0.408551
2,A0A087WTH1_1_67,A0A087WTH1,67,0.532,10.160125,6.069995,0.822738,0.199126,2.243991,5.283533,...,0.384885,2.281943,1.092688,0.298507,0.537,10.167,20.524202,1.094369,5.764964,0.403545
3,A0A087WTH5_1_132,A0A087WTH5,132,0.496,10.049124,6.150972,0.798438,0.188978,3.034005,7.119714,...,0.799692,0.000000,0.627563,0.000000,0.509,10.018,28.732651,1.057445,5.766837,0.365432
4,A0A087WV53_1_46,A0A087WV53,46,0.567,10.212765,6.439589,0.882210,0.214786,1.882622,4.607908,...,1.947253,0.000000,0.090132,0.369565,0.578,10.316,17.085913,1.100872,6.130355,0.438200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28053,Q9Y6Z4_1_181,Q9Y6Z4,181,0.493,10.006512,5.748098,0.799905,0.210890,3.791956,8.607822,...,-1.319891,0.000000,0.163169,0.000000,0.520,9.993,33.590806,1.128366,5.388851,0.391268
28054,Q9Y6Z5_1_254,Q9Y6Z5,254,0.528,9.830710,6.153759,0.896173,0.221422,4.586832,10.808776,...,-2.182869,0.000000,2.214949,0.000000,0.537,9.984,39.658059,1.156510,5.803936,0.394400
28055,Q9Y6Z7_1_109,Q9Y6Z7,109,0.546,10.025622,6.175505,0.935857,0.216970,3.011145,7.113405,...,0.451957,2.132808,1.114749,0.513761,0.513,10.072,25.880343,1.164103,5.738483,0.426205
28056,W6CW81_1_113,W6CW81,113,0.519,10.084249,6.191149,0.843169,0.193880,2.943990,6.975229,...,0.554013,2.234693,-1.942090,0.451327,0.512,10.063,26.561695,1.108354,5.813349,0.400844


# write function

In [10]:
def compute_3dplot_from_seq_name(seq_name):
    global protein_df
    example_protein_dir = seq_name_dir_df[seq_name_dir_df.seq_name==seq_name].seq_dir.values[0]
    protein_rg_theta = seq_name_AFRC[seq_name_AFRC.seq_name==seq_name].AFRC_mean_rg_theta.values[0]
    #convert nm to angstrom for lindorff-larsen data
    protein_rg_by_rg_theta = (np.load(f'{example_protein_dir}/rg.npy')*10)/protein_rg_theta
    protein_df = pd.DataFrame(zip(np.load(f'{example_protein_dir}/rg.npy')**2,
                                 protein_rg_by_rg_theta),columns=['Rg2','Rg/Rg_theta']).copy()
    protein_df['Ree2'] = np.load(f'{example_protein_dir}/ete.npy')**2
    protein_df['ratio'] = protein_df['Ree2']/protein_df['Rg2']
    return protein_3dplot_against_GW(protein_df,seq_name,'protein','magenta')

In [11]:
def RSA_Rs_compute_3dplot_from_seq_name(seq_name):
    global protein_df
    example_protein_dir = seq_name_dir_df[seq_name_dir_df.seq_name==seq_name].seq_dir.values[0]
    t = md.load(f'{example_protein_dir}/traj.xtc', top=f'{example_protein_dir}/top.pdb')
    t_df_moments = pd.DataFrame(md.principal_moments(t),columns=['R3','R2','R1']).iloc[10:,:].copy()
    t_df_moments['asphericity']=t_df_moments.R1.values-(0.5*(t_df_moments.R2.values+t_df_moments.R3.values))
    t_df_moments['acylindricity']=t_df_moments.R2.values-t_df_moments.R3.values
    t_df_moments['RSA']=(t_df_moments.asphericity.values**2+(0.75*t_df_moments.acylindricity.values**2))/(t_df_moments.R1.values+t_df_moments.R2.values+t_df_moments.R3.values)**2
    protein_df = t_df_moments[['RSA']].copy()
    del t_df_moments
    protein_df['Rg2'] = np.load(f'{example_protein_dir}/rg.npy')**2
    protein_df['Ree2'] = np.load(f'{example_protein_dir}/ete.npy')**2
    protein_df['ratio'] = protein_df['Ree2']/protein_df['Rg2']
    return RSA_Rs_protein_3dplot_against_GW(protein_df,seq_name,'protein','magenta')

In [12]:
def load_trajectory(seq_name):
    #https://soursop.readthedocs.io/en/latest/usage/overview.html
    from soursop.sstrajectory import SSTrajectory
    global t_md, t_soursop,ree
    example_protein_dir = seq_name_dir_df[seq_name_dir_df.seq_name==seq_name].seq_dir.values[0]
    t_md = md.load(f'../larsen_paper_2023_compaction/{example_protein_dir}/traj.xtc', 
                   top=f'../larsen_paper_2023_compaction/{example_protein_dir}/top.pdb')
    t_soursop = SSTrajectory(f'../larsen_paper_2023_compaction/{example_protein_dir}/traj.xtc',
                             f'../larsen_paper_2023_compaction/{example_protein_dir}/top.pdb')
    ree = np.load(f'../larsen_paper_2023_compaction/{example_protein_dir}/ete.npy')
    ProtObj = t_soursop.proteinTrajectoryList[0]
    return ProtObj


In [13]:
#Rij is from lindorff-larsen
def Rij(traj):
    pairs = traj.top.select_pairs('all','all')
    d = md.compute_distances(traj,pairs)
    nres = traj.n_atoms
    ij = np.arange(2,nres,1)
    diff = [x[1]-x[0] for x in pairs]
    dij = np.empty(0)
    for i in ij:
        dij = np.append(dij,np.sqrt((d[:,diff==i]**2).mean().mean()))
    f = lambda x,R0,v : R0*np.power(x,v)
    popt, pcov = curve_fit(f,ij[ij>5],dij[ij>5],p0=[.4,.5])
    nu = popt[1]
    nu_err = pcov[1,1]**0.5
    R0 = popt[0]
    R0_err = pcov[0,0]**0.5
    return ij,dij,nu,nu_err,R0,R0_err

def calculate_nu_KLL_from_seq_name(seq_name, start_residue, end_residue):
    example_protein_dir = seq_name_dir_df[seq_name_dir_df.seq_name == seq_name].seq_dir.values[0]
    t_md = md.load(f'../larsen_paper_2023_compaction/{example_protein_dir}/traj.xtc', 
                   top=f'../larsen_paper_2023_compaction/{example_protein_dir}/top.pdb')
    # Skipping the first 10 frames
    t_md = t_md[10:]
    subsequence_indices = t_md.topology.select(f'residue {start_residue-1} to {end_residue-1}')
    
    subsequence_traj = t_md.atom_slice(subsequence_indices)

    ij, dij, nu, nu_err, R0, R0_err = Rij(subsequence_traj)
    return nu, nu_err

#original lindorff-larsen code,keeping as reference (Rij is original function from their github)
# def calculate_nu_KLL(seq_name):
#     example_protein_dir = seq_name_dir_df[seq_name_dir_df.seq_name==seq_name].seq_dir.values[0]
#     t_md = md.load(f'../larsen_paper_2023_compaction/{example_protein_dir}/traj.xtc', 
#                    top=f'../larsen_paper_2023_compaction/{example_protein_dir}/top.pdb')
#     ij,dij,nu,nu_err,R0,R0_err = Rij(t_md)
#     return nu

# def Rij(traj):
#     pairs = traj.top.select_pairs('all','all')
#     d = md.compute_distances(traj,pairs)
#     nres = traj.n_atoms
#     ij = np.arange(2,nres,1)
#     diff = [x[1]-x[0] for x in pairs]
#     dij = np.empty(0)
#     for i in ij:
#         dij = np.append(dij,np.sqrt((d[:,diff==i]**2).mean().mean()))
#     f = lambda x,R0,v : R0*np.power(x,v)
#     popt, pcov = curve_fit(f,ij[ij>5],dij[ij>5],p0=[.4,.5])
#     nu = popt[1]
#     nu_err = pcov[1,1]**0.5
#     R0 = popt[0]
#     R0_err = pcov[0,0]**0.5
#     return ij,dij,nu,nu_err,R0,R0_err

In [14]:
#master_df = master_df.reset_index(drop=True).copy()

# use  pyconformap shape vs size

In [15]:
exec(open("../larsen_paper_2023_compaction/pyconformap_modified.py").read())
def generate_2d_map(seq_name):
    location_of_files = seq_name_dir_df[seq_name_dir_df.seq_name==seq_name].seq_dir.values[0]    
    rg2_value = np.load(location_of_files+'/rg.npy')**2
    ree2_value = np.load(location_of_files+'/ete.npy')**2
    pd.DataFrame(zip(rg2_value,ree2_value),columns=['Rg2','Ree2']).to_csv(seq_name+'.csv',index=False)
    map_2d_seq_name = PyConforMap(seq_name+'.csv')
    map_2d_seq_name.plot_protein_against_GW(seq_name+'.csv')
    os.remove(seq_name+'.csv')
    return 

# combine the df


In [16]:
seq_name_fluctations = pd.read_csv('../larsen_paper_2023_compaction/HPC_computed_fC_values_all.csv').set_index('seq_name_list')
seq_stdev = pd.read_csv('../holehouse_project/IDRome_Rg_Rs_RSA_stdev.csv').set_index('seq_name')
seq_ALBATROSS = pd.read_csv('../holehouse_project/IDRome_with_ALBATROSS_calculations.csv').set_index('seq_name')
seq_ranges = pd.read_csv('../holehouse_project/IDRome_Rg_Rs_RSA_range.csv').set_index('seq_name')
# the bounded_frac_size_shape is only for size-shape (through pyconformap_modified)


#add fP to the property df
seq_name_AFRC['fP'] = [seq.count('P')/len(seq) for seq in seq_name_AFRC.fasta.values]

#recalculate FCR
seq_name_AFRC['net_charge'] = [(seq.count('K')+seq.count('R')-seq.count('D')-seq.count('E'))/len(seq) for seq in seq_name_AFRC.fasta.values]

idrome_prop_flucs = pd.concat([seq_name_AFRC.set_index('seq_name'),
           seq_name_fluctations[['fC_shape_shape',
                                 'fA_shape_shape',
                                 'fC_size_shape',
                                 'fA_size_shape',
                                 'bounded_frac_size_shape']],
                              seq_stdev[['Rg_std','Rs_std','RSA_std']],
                              seq_ALBATROSS[['albatross_Rg','albatross_Rg_scaled','albatross_Ree','albatross_Ree_scaled',
                                             'albatross_scaling_exponent']],
                              seq_ranges[['Rg_range','Rs_range','RSA_range']]],
          axis=1).reset_index().rename(columns={'index':'seq_name'}).copy()

del seq_name_fluctations, seq_name_AFRC, seq_stdev, seq_ALBATROSS
idrome_prop_flucs

,seq_name,UniProt_ID,N,nu,SPR,ete2_Rg2,S,Delta,Rg/nm,Ree/nm,...,Rs_std,RSA_std,albatross_Rg,albatross_Rg_scaled,albatross_Ree,albatross_Ree_scaled,albatross_scaling_exponent,Rg_range,Rs_range,RSA_range
0,A0A024RBG1_145_181,A0A024RBG1,37,0.558,10.284649,6.304027,0.933961,0.225950,1.606561,3.843011,...,2.620898,0.200553,1.737755,1.737755,4.138554,4.251985,0.578297,0.790029,7.039022,0.537698
1,A0A075B6T7_1_32,A0A075B6T7,32,0.525,10.092351,6.114494,0.901807,0.212670,1.448141,3.390487,...,2.737865,0.201459,1.524150,1.524150,3.321804,3.321804,0.498650,0.756501,7.359276,0.538843
2,A0A087WTH1_1_67,A0A087WTH1,67,0.532,10.160125,6.069995,0.822738,0.199126,2.243991,5.283533,...,2.764873,0.181605,2.407502,2.407502,5.670386,5.788718,0.544833,1.185494,7.414943,0.500908
3,A0A087WTH5_1_132,A0A087WTH5,132,0.496,10.049124,6.150972,0.798438,0.188978,3.034005,7.119714,...,3.071942,0.183702,3.189458,3.189458,7.614085,7.813105,0.504434,1.620033,8.038966,0.485090
4,A0A087WV53_1_46,A0A087WV53,46,0.567,10.212765,6.439589,0.882210,0.214786,1.882622,4.607908,...,2.559872,0.187474,1.984436,1.984436,4.691561,4.962079,0.566668,0.945399,6.813072,0.499202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28053,Q9Y6Z4_1_181,Q9Y6Z4,181,0.493,10.006512,5.748098,0.799905,0.210890,3.791956,8.607822,...,2.880116,0.186104,4.129234,4.129234,9.571439,9.803478,0.534929,2.045043,7.521686,0.492647
28054,Q9Y6Z5_1_254,Q9Y6Z5,254,0.528,9.830710,6.153759,0.896173,0.221422,4.586832,10.808776,...,3.086350,0.192582,4.447731,4.447731,10.693898,10.582268,0.504626,2.664859,7.982766,0.519563
28055,Q9Y6Z7_1_109,Q9Y6Z7,109,0.546,10.025622,6.175505,0.935857,0.216970,3.011145,7.113405,...,2.783294,0.192728,3.285178,3.285178,7.447835,7.863577,0.570739,1.685618,7.465008,0.516141
28056,W6CW81_1_113,W6CW81,113,0.519,10.084249,6.191149,0.843169,0.193880,2.943990,6.975229,...,2.866177,0.187885,3.052207,3.052207,7.340392,7.289317,0.541635,1.597735,7.462810,0.493407


In [17]:
idrome_prop_flucs.columns

Index(['seq_name', 'UniProt_ID', 'N', 'nu', 'SPR', 'ete2_Rg2', 'S', 'Delta',
       'Rg/nm', 'Ree/nm', 'Rh/nm', 'fK', 'fR', 'fE', 'fD', 'faro',
       'mean_lambda', 'shd', 'scd', 'kappa', 'fcr', 'ncpr', 'fasta',
       'is_btw_folded', 'is_nterm', 'is_cterm', 'is_idp', 'first', 'last',
       'N_FL', 'protein_name', 'gene_name', 'QCDpred', 'z_delta_+-',
       'z_omega_pi', 'z_omega_+', 'z_omega_-', 'z_omega_h', 'fdomain',
       'nu_svr', 'SPR_svr', 'AFRC_mean_rg_theta', 'rg_rg_theta_mean',
       'mean_inst_ratio', 'mean_RSA', 'fP', 'net_charge', 'fC_shape_shape',
       'fA_shape_shape', 'fC_size_shape', 'fA_size_shape',
       'bounded_frac_size_shape', 'Rg_std', 'Rs_std', 'RSA_std',
       'albatross_Rg', 'albatross_Rg_scaled', 'albatross_Ree',
       'albatross_Ree_scaled', 'albatross_scaling_exponent', 'Rg_range',
       'Rs_range', 'RSA_range'],
      dtype='object')

### DO NOT RUN the immediately below cell

In [16]:
#DO NOT run this cell
#these FOUR IDR simulations have low snapshots (<1000)
# %%time
# for seq_dir in seq_name_dir_df.seq_dir.values:
#     if np.load(seq_dir+'/rg.npy').shape[0]!=1000:
#         print(seq_name_dir_df[seq_name_dir_df.seq_dir==seq_dir].seq_name, np.load(seq_dir+'/rg.npy').shape[0])

3703    Q53SF7_218_1128
Name: seq_name, dtype: object 500
11700    Q7Z2Y5_341_1224
Name: seq_name, dtype: object 420
23881    Q9Y2W1_1_611
Name: seq_name, dtype: object 100
27365    Q9BXT5_1_968
Name: seq_name, dtype: object 100
CPU times: user 3.77 s, sys: 2.96 s, total: 6.73 s
Wall time: 14min 48s


In [18]:
#drop these FOUR IDR simulations because of low snapshots #
seq_name_dir_df = seq_name_dir_df[~seq_name_dir_df.seq_name.isin(['Q53SF7_218_1128',
                                             'Q7Z2Y5_341_1224',
                                             'Q9Y2W1_1_611',
                                             'Q9BXT5_1_968'])].copy()

idrome_prop_flucs = idrome_prop_flucs[~idrome_prop_flucs.seq_name.isin(['Q53SF7_218_1128',
                                             'Q7Z2Y5_341_1224',
                                             'Q9Y2W1_1_611',
                                             'Q9BXT5_1_968'])].copy()

## additional functions for visualization

In [19]:
def compute_30mer_data_from_seq_name(seq_name,k_frac, h5_file_path):
    print('using 30 as moving window size, constant, based on protein of smallest length')
    print('for proteins of length more than 60, moving window size = 30')
    print('for proteins of length less than or equal to 60, moving window size = 1/k_frac of length')
    example_protein_dir = seq_name_dir_df[seq_name_dir_df.seq_name==seq_name].seq_dir.values[0]
    
    traj = md.load(f'../larsen_paper_2023_compaction/{example_protein_dir}/traj.xtc',
                   top=f'../larsen_paper_2023_compaction/{example_protein_dir}/top.pdb')
    
    fasta_sequence = idrome_prop_flucs[idrome_prop_flucs.seq_name==seq_name].fasta.values[0]
    
    # Skipping the first 10 frames
    traj = traj[10:]    
    
    # Set the subsequence length k (replace 4 with your desired value)
    n_residues = traj.topology.n_residues

    if len(fasta_sequence) <= 60:
        k = round(traj.topology.n_residues/k_frac)
    elif len(fasta_sequence) > 60:
        k = 30

    with h5py.File(h5_file_path, 'a') as h5file:
        protein_group = h5file.create_group(seq_name)

        # Compute various properties for the whole protein
        complete_protein_rgyr = np.mean(md.compute_rg(traj))
        protein_group.create_dataset('full_protein_rgyr', data=complete_protein_rgyr)

        # store fasta sequence
        protein_group.create_dataset('fasta_sequence_whole_protein', data=fasta_sequence)

        # store seq_name
        protein_group.create_dataset('seq_name', data=seq_name)
        
        first_bead_index = traj.topology.select(f"residue {0}")[0]
        last_bead_index = traj.topology.select(f"residue {n_residues - 1}")[0]    
        end_to_end_distances = md.compute_distances(traj, [[first_bead_index, last_bead_index]])
        end_to_end_distances = end_to_end_distances.flatten()
        complete_protein_ete = end_to_end_distances
        complete_protein_inst_ratio = np.mean((complete_protein_ete**2)/(complete_protein_rgyr**2))
        protein_group.create_dataset('full_protein_ratio', data=complete_protein_inst_ratio)
    
        complete_protein_moments = pd.DataFrame(md.principal_moments(traj),columns=['R3','R2','R1']).copy()
        complete_protein_moments['asphericity']=complete_protein_moments.R1.values-(0.5*(complete_protein_moments.R2.values+complete_protein_moments.R3.values))
        complete_protein_moments['acylindricity']=complete_protein_moments.R2.values-complete_protein_moments.R3.values
        complete_protein_moments['RSA']=(complete_protein_moments.asphericity.values**2+(0.75*complete_protein_moments.acylindricity.values**2))/(complete_protein_moments.R1.values+complete_protein_moments.R2.values+complete_protein_moments.R3.values)**2
        complete_protein_RSA = np.mean(complete_protein_moments['RSA'].values)    
        protein_group.create_dataset('full_protein_RSA', data=complete_protein_RSA)

        #AFRC
        complete_protein_afrc_init = afrc.AnalyticalFRC(fasta_sequence)
        complete_protein_rg_theta_mean = complete_protein_afrc_init.get_mean_radius_of_gyration()
        complete_protein_rg_rg_theta_mean = np.mean((10*md.compute_rg(traj))/complete_protein_rg_theta_mean)
        protein_group.create_dataset('full_protein_rg_rg_theta_mean', data=complete_protein_rg_rg_theta_mean)

        #nu
        complete_protein_nu = calculate_nu_KLL_from_seq_name(seq_name, 1, n_residues)[0]
        complete_protein_nu_err = calculate_nu_KLL_from_seq_name(seq_name, 1, n_residues)[1]
        protein_group.create_dataset('full_protein_nu_recompute', data=complete_protein_nu)
        protein_group.create_dataset('full_protein_nu_recompute_err', data=complete_protein_nu_err)
    
        j = 0
        seq_name_list = []
        k_list = []
        mid_residue = []
        subchain_mean_Rg = []
        subchain_mean_Rs = []
        subchain_mean_RSA = []
        subchain_mean_rg_rg_theta = []
        subchain_nu = []
        
        # Iterate through each subsequence of k residues
        for start_res in range(1, n_residues - k + 2):  # Ensures we don't go out of bounds
            seq_name_list.append(seq_name)
            k_list.append(k)
            # Select k consecutive residues
            subsequence_group = protein_group.create_group(f'mer_{start_res}_{start_res + k - 1}')
            selection_string = f"residue {start_res-1} to {start_res + k - 2}"  # MDTraj is zero-indexed

            mid_residue.append(((start_res-1) + (start_res + k - 2) ) //2 )

            
            subsequence_indices = traj.topology.select(selection_string)
            
            fasta_slice = fasta_sequence[(start_res-1):(start_res + k - 1)]
            subsequence_group.create_dataset('fasta_subsequence', data=fasta_slice)
            subsequence_group.create_dataset('selection_string', data=selection_string)
            
            
            # Create a trajectory slice for the selected subsequence
            subsequence_traj = traj.atom_slice(subsequence_indices)
            
            # Calculate the radius of gyration for the subsequence over all remaining frames
            rgyr = md.compute_rg(subsequence_traj)
            subsequence_group.create_dataset('Rg/nm', data=rgyr)
            subchain_mean_Rg.append(np.mean(rgyr))
            
            #calculate nu
            subsequence_nu = calculate_nu_KLL_from_seq_name(seq_name, start_res, start_res + k - 1)[0]
            subsequence_nu_err = calculate_nu_KLL_from_seq_name(seq_name, start_res, start_res + k - 1)[1]
            subsequence_group.create_dataset('nu_recompute', data=subsequence_nu)
            subsequence_group.create_dataset('nu_recompute_err', data=subsequence_nu_err)
            subchain_nu.append(subsequence_nu)
            
            # Select indices for the first and last bead in the subsequence for end-to-end distance calculation
            first_bead_index = traj.topology.select(f"residue {start_res-1}")[0]
            last_bead_index = traj.topology.select(f"residue {start_res + k - 2}")[0]
        
            # Calculate end-to-end distances for the subsequence over all remaining frames
            end_to_end_distances = md.compute_distances(traj, [[first_bead_index, last_bead_index]])
            end_to_end_distances = end_to_end_distances.flatten()
            subsequence_group.create_dataset('ete', data=end_to_end_distances)
            
            running_inst_ratio = (end_to_end_distances**2)/(rgyr**2)
            subsequence_group.create_dataset('inst_ratio', data=running_inst_ratio)
            subchain_mean_Rs.append(np.mean(running_inst_ratio))
            
            t_df_moments = pd.DataFrame(md.principal_moments(subsequence_traj),columns=['R3','R2','R1']).copy()
            t_df_moments['asphericity']=t_df_moments.R1.values-(0.5*(t_df_moments.R2.values+t_df_moments.R3.values))
            t_df_moments['acylindricity']=t_df_moments.R2.values-t_df_moments.R3.values
            t_df_moments['RSA']=(t_df_moments.asphericity.values**2+(0.75*t_df_moments.acylindricity.values**2))/(t_df_moments.R1.values+t_df_moments.R2.values+t_df_moments.R3.values)**2
            subsequence_group.create_dataset('RSA', data=t_df_moments['RSA'].values)
            subchain_mean_RSA.append(np.mean(t_df_moments['RSA'].values))
    
            #AFRC
            afrc_init = afrc.AnalyticalFRC(fasta_slice)
            rg_theta_mean = afrc_init.get_mean_radius_of_gyration()
            rg_rg_theta_mean = (10*rgyr) / rg_theta_mean
            subsequence_group.create_dataset('rg_rg_theta_mean', data=rg_rg_theta_mean)
            subchain_mean_rg_rg_theta.append(np.mean(rg_rg_theta_mean))
        global combined_subchain_metrics
        combined_subchain_metrics = pd.DataFrame(zip(seq_name_list,k_list,mid_residue,subchain_mean_Rg,subchain_mean_Rs,subchain_mean_RSA,
                                                subchain_mean_rg_rg_theta,subchain_nu),
                                             columns = ['seq_name','window_size','mid_residue','mean_Rg','mean_Rs','mean_RSA',
                                                'mean_rg_rg_theta','nu'])
        #Rg metrics
        min_mean_Rg = [combined_subchain_metrics.mean_Rg.min()]
        mid_res_min_mean_Rg = [combined_subchain_metrics[combined_subchain_metrics.mean_Rg == combined_subchain_metrics.mean_Rg.min()].mid_residue.values[0]]
        max_mean_Rg = [combined_subchain_metrics.mean_Rg.max()]
        mid_res_max_mean_Rg = [combined_subchain_metrics[combined_subchain_metrics.mean_Rg == combined_subchain_metrics.mean_Rg.max()].mid_residue.values[0]]
        stdev_mean_Rg = [combined_subchain_metrics.mean_Rg.std()]

        #Rs metrics
        min_mean_Rs = [combined_subchain_metrics.mean_Rs.min()]
        mid_res_min_mean_Rs = [combined_subchain_metrics[combined_subchain_metrics.mean_Rs == combined_subchain_metrics.mean_Rs.min()].mid_residue.values[0]]
        max_mean_Rs = [combined_subchain_metrics.mean_Rs.max()]
        mid_res_max_mean_Rs = [combined_subchain_metrics[combined_subchain_metrics.mean_Rs == combined_subchain_metrics.mean_Rs.max()].mid_residue.values[0]]
        stdev_mean_Rs = [combined_subchain_metrics.mean_Rs.std()]
        
        #RSA metrics
        min_mean_RSA = [combined_subchain_metrics.mean_RSA.min()]
        mid_res_min_mean_RSA = [combined_subchain_metrics[combined_subchain_metrics.mean_RSA == combined_subchain_metrics.mean_RSA.min()].mid_residue.values[0]]
        max_mean_RSA = [combined_subchain_metrics.mean_RSA.max()]
        mid_res_max_mean_RSA = [combined_subchain_metrics[combined_subchain_metrics.mean_RSA == combined_subchain_metrics.mean_RSA.max()].mid_residue.values[0]]
        stdev_mean_RSA = [combined_subchain_metrics.mean_RSA.std()]
        
        #rg_rg_theta metrics
        min_mean_rg_rg_theta = [combined_subchain_metrics.mean_rg_rg_theta.min()]
        mid_res_min_mean_rg_rg_theta = [combined_subchain_metrics[combined_subchain_metrics.mean_rg_rg_theta == combined_subchain_metrics.mean_rg_rg_theta.min()].mid_residue.values[0]]
        max_mean_rg_rg_theta = [combined_subchain_metrics.mean_rg_rg_theta.max()]
        mid_res_max_mean_rg_rg_theta = [combined_subchain_metrics[combined_subchain_metrics.mean_rg_rg_theta == combined_subchain_metrics.mean_rg_rg_theta.max()].mid_residue.values[0]]
        stdev_mean_rg_rg_theta = [combined_subchain_metrics.mean_rg_rg_theta.std()]
        
        #nu metrics
        min_mean_nu = [combined_subchain_metrics.nu.min()]
        mid_res_min_mean_nu = [combined_subchain_metrics[combined_subchain_metrics.nu == combined_subchain_metrics.nu.min()].mid_residue.values[0]]
        max_mean_nu = [combined_subchain_metrics.nu.max()]
        mid_res_max_mean_nu = [combined_subchain_metrics[combined_subchain_metrics.nu == combined_subchain_metrics.nu.max()].mid_residue.values[0]]
        stdev_mean_nu = [combined_subchain_metrics.nu.std()]
        
        #generate dataframe of chosen subchain metrics
        selected_subchain_metrics = pd.DataFrame(zip([combined_subchain_metrics.seq_name.unique()[0]],[combined_subchain_metrics.window_size.unique()[0]],
            min_mean_Rg,mid_res_min_mean_Rg,max_mean_Rg,mid_res_max_mean_Rg,stdev_mean_Rg,
                                                    min_mean_Rs,mid_res_min_mean_Rs,max_mean_Rs,mid_res_max_mean_Rs,stdev_mean_Rs,
                                                    min_mean_RSA,mid_res_min_mean_RSA,max_mean_RSA,mid_res_max_mean_RSA,stdev_mean_RSA,
                        min_mean_rg_rg_theta,mid_res_min_mean_rg_rg_theta,max_mean_rg_rg_theta,mid_res_max_mean_rg_rg_theta,stdev_mean_rg_rg_theta,
                                            min_mean_nu,mid_res_min_mean_nu,max_mean_nu,mid_res_max_mean_nu,stdev_mean_nu),
                                                 columns=['seq_name','window_size',
                                                 'min_mean_Rg','mid_res_min_mean_Rg','max_mean_Rg','mid_res_max_mean_Rg','stdev_mean_Rg',
                                                         'min_mean_Rs','mid_res_min_mean_Rs','max_mean_Rs','mid_res_max_mean_Rs','stdev_mean_Rs',
                                                'min_mean_RSA','mid_res_min_mean_RSA','max_mean_RSA','mid_res_max_mean_RSA','stdev_mean_RSA',
            'min_mean_rg_rg_theta','mid_res_min_mean_rg_rg_theta','max_mean_rg_rg_theta','mid_res_max_mean_rg_rg_theta','stdev_mean_rg_rg_theta',
                                            'min_mean_nu','mid_res_min_mean_nu','max_mean_nu','mid_res_max_mean_nu','stdev_mean_nu'])
        print('ALL DONE')    
    return selected_subchain_metrics

In [20]:
seq_name_chunks = np.array_split(idrome_prop_flucs.seq_name.values, 100)
j=0
for i, chunk in enumerate(seq_name_chunks):
    print(f"Chunk {i+1}: {chunk.shape[0]}")
    j+=chunk.shape[0]
    print(f' {j}')

Chunk 1: 281
 281
Chunk 2: 281
 562
Chunk 3: 281
 843
Chunk 4: 281
 1124
Chunk 5: 281
 1405
Chunk 6: 281
 1686
Chunk 7: 281
 1967
Chunk 8: 281
 2248
Chunk 9: 281
 2529
Chunk 10: 281
 2810
Chunk 11: 281
 3091
Chunk 12: 281
 3372
Chunk 13: 281
 3653
Chunk 14: 281
 3934
Chunk 15: 281
 4215
Chunk 16: 281
 4496
Chunk 17: 281
 4777
Chunk 18: 281
 5058
Chunk 19: 281
 5339
Chunk 20: 281
 5620
Chunk 21: 281
 5901
Chunk 22: 281
 6182
Chunk 23: 281
 6463
Chunk 24: 281
 6744
Chunk 25: 281
 7025
Chunk 26: 281
 7306
Chunk 27: 281
 7587
Chunk 28: 281
 7868
Chunk 29: 281
 8149
Chunk 30: 281
 8430
Chunk 31: 281
 8711
Chunk 32: 281
 8992
Chunk 33: 281
 9273
Chunk 34: 281
 9554
Chunk 35: 281
 9835
Chunk 36: 281
 10116
Chunk 37: 281
 10397
Chunk 38: 281
 10678
Chunk 39: 281
 10959
Chunk 40: 281
 11240
Chunk 41: 281
 11521
Chunk 42: 281
 11802
Chunk 43: 281
 12083
Chunk 44: 281
 12364
Chunk 45: 281
 12645
Chunk 46: 281
 12926
Chunk 47: 281
 13207
Chunk 48: 281
 13488
Chunk 49: 281
 13769
Chunk 50: 281
 140

In [21]:
test_metrics = pd.read_csv('TESTING_parallelization_store_all_subchains_Jesse_moving_window_HDF5.csv')
test_metrics

,Unnamed: 0,seq_name,window_size,min_mean_Rg,mid_res_min_mean_Rg,max_mean_Rg,mid_res_max_mean_Rg,stdev_mean_Rg,min_mean_Rs,mid_res_min_mean_Rs,...,min_mean_rg_rg_theta,mid_res_min_mean_rg_rg_theta,max_mean_rg_rg_theta,mid_res_max_mean_rg_rg_theta,stdev_mean_rg_rg_theta,min_mean_nu,mid_res_min_mean_nu,max_mean_nu,mid_res_max_mean_nu,stdev_mean_nu
0,0,A0A024RBG1_145_181,12,0.787629,6,0.828673,19,0.015109,6.321465,28,...,0.913884,8,0.954280,18,0.014482,0.604528,13,0.655475,18,0.013469
1,0,A0A075B6T7_1_32,11,0.748416,24,0.789972,12,0.012647,6.131754,19,...,0.908635,24,0.962646,12,0.014967,0.575042,18,0.645993,7,0.019511
2,0,A0A087WTH1_1_67,30,1.406829,19,1.472289,51,0.018825,5.698813,14,...,1.022500,19,1.073180,51,0.013978,0.524312,19,0.592436,47,0.024612
3,0,A0A087WTH5_1_132,30,1.386801,35,1.498327,116,0.033564,5.529051,51,...,1.015959,57,1.093254,116,0.021404,0.518506,50,0.587438,113,0.019152
4,0,A0A087WV53_1_46,15,0.930813,15,0.985087,7,0.016192,5.840974,31,...,0.955543,31,1.010013,22,0.017341,0.553039,32,0.645676,20,0.029551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,0,A2IDD5_317_438,30,1.379039,91,1.513469,21,0.037775,5.609603,100,...,1.009741,91,1.104647,22,0.026007,0.529038,100,0.585359,20,0.012630
277,0,A2IDD5_102_166,30,1.422826,14,1.496717,46,0.016616,6.014925,49,...,1.040120,14,1.097931,47,0.013850,0.564297,14,0.590320,20,0.007207
278,0,A2PYH4_1091_1435,30,1.397212,201,1.651645,176,0.045287,5.477315,197,...,1.026226,202,1.207664,176,0.033766,0.514635,197,0.655160,174,0.026100
279,0,A2PYH4_1_266,30,1.406721,176,1.591095,41,0.044717,5.791986,49,...,1.024078,225,1.159552,41,0.032804,0.540005,176,0.646296,39,0.021089


In [24]:
seq_name_chunks[0] 

array(['A0A024RBG1_145_181', 'A0A075B6T7_1_32', 'A0A087WTH1_1_67',
       'A0A087WTH5_1_132', 'A0A087WV53_1_46', 'A0A087WVF3_214_549',
       'A0A087WVF3_1_53', 'A0A087WWA1_1_95', 'A0A087WX78_1_165',
       'A0A087WX78_291_387', 'A0A087WXM9_1_373', 'A0A087WXS9_1_50',
       'A0A087WXS9_214_549', 'A0A087X179_1_50', 'A0A087X179_214_549',
       'A0A087X1G2_214_549', 'A0A087X1G2_1_53', 'A0A096LNP1_1_70',
       'A0A096LP01_37_95', 'A0A096LP49_1_634', 'A0A096LP49_681_1063',
       'A0A096LPI5_1_108', 'A0A0A0MT76_1_42', 'A0A0A6YYL3_331_461',
       'A0A0A6YYL3_1_93', 'A0A0B4J2D5_1_36', 'A0A0B4J2F2_350_783',
       'A0A0B4J2F2_278_307', 'A0A0D9SF12_1_145', 'A0A0G2JMD5_225_283',
       'A0A0G2JMD5_106_149', 'A0A0J9YWL9_151_993', 'A0A0J9YX94_426_578',
       'A0A0J9YX94_76_274', 'A0A0J9YXQ4_85_350', 'A0A0J9YXQ4_487_647',
       'A0A0J9YY54_1_62', 'A0A0J9YY54_151_714', 'A0A0K2S4Q6_130_201',
       'A0A0U1RQF7_1_123', 'A0A0U1RQI7_1_968', 'A0A0U1RQS6_1_177',
       'A0A0U1RR11_239_272', 'A0A0U1RR

In [11]:
5610*1

5610

In [12]:
5610*2

11220

In [13]:
5610*3

16830

In [10]:
5610*4

22440

In [14]:
5610*5

28050

## some tests

In [93]:
%%time
test = compute_30mer_data_from_seq_name('A0A087WTH5_1_132',3, 'test_folder/test.h5')

using 30 as moving window size, constant, based on protein of smallest length
for proteins of length more than 60, moving window size = 30
for proteins of length less than or equal to 60, moving window size = 1/k_frac of length
ALL DONE
CPU times: user 14.8 s, sys: 158 ms, total: 14.9 s
Wall time: 15 s


In [94]:
combined_subchain_metrics.mid_residue.values

array([ 14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116])

In [67]:
def orig_compute_kmer_data_from_seq_name(seq_name):
    print('this is the original say to get kmer data from seq name, NO HDF5')
    example_protein_dir = seq_name_dir_df[seq_name_dir_df.seq_name==seq_name].seq_dir.values[0]
    
    traj = md.load(f'../larsen_paper_2023_compaction/{example_protein_dir}/traj.xtc',
                   top=f'../larsen_paper_2023_compaction/{example_protein_dir}/top.pdb')
    
    fasta_sequence = idrome_prop_flucs[idrome_prop_flucs.seq_name==seq_name].fasta.values[0]
    
    # Skipping the first 10 frames
    traj = traj[10:]    
    
    # Set the subsequence length k (replace 4 with your desired value)
    n_residues = traj.topology.n_residues
    k = 30

    complete_protein_rgyr = np.mean(md.compute_rg(traj))
    complete_protein_nu = calculate_nu_KLL_from_seq_name(seq_name, 1, n_residues)[0]
    complete_protein_nu_err = calculate_nu_KLL_from_seq_name(seq_name, 1, n_residues)[1]
    first_bead_index = traj.topology.select(f"residue {0}")[0]
    last_bead_index = traj.topology.select(f"residue {n_residues - 1}")[0]    
    end_to_end_distances = md.compute_distances(traj, [[first_bead_index, last_bead_index]])
    end_to_end_distances = end_to_end_distances.flatten()
    complete_protein_ete = end_to_end_distances
    complete_protein_inst_ratio = np.mean((complete_protein_ete**2)/(complete_protein_rgyr**2))
    
    complete_protein_moments = pd.DataFrame(md.principal_moments(traj),columns=['R3','R2','R1']).copy()
    complete_protein_moments['asphericity']=complete_protein_moments.R1.values-(0.5*(complete_protein_moments.R2.values+complete_protein_moments.R3.values))
    complete_protein_moments['acylindricity']=complete_protein_moments.R2.values-complete_protein_moments.R3.values
    complete_protein_moments['RSA']=(complete_protein_moments.asphericity.values**2+(0.75*complete_protein_moments.acylindricity.values**2))/(complete_protein_moments.R1.values+complete_protein_moments.R2.values+complete_protein_moments.R3.values)**2
    complete_protein_RSA = np.mean(complete_protein_moments['RSA'].values)    

    #AFRC
    complete_protein_afrc_init = afrc.AnalyticalFRC(fasta_sequence)
    complete_protein_rg_theta_mean = complete_protein_afrc_init.get_mean_radius_of_gyration()
    complete_protein_rg_rg_theta_mean = np.mean((10*md.compute_rg(traj))/complete_protein_rg_theta_mean)

    
    j = 0
    # Iterate through each subsequence of k residues
    for start_res in range(1, n_residues - k + 2):  # Ensures we don't go out of bounds
        # Select k consecutive residues
        selection_string = f"residue {start_res-1} to {start_res + k - 2}"  # MDTraj is zero-indexed
        subsequence_indices = traj.topology.select(selection_string)
        
        fasta_slice = fasta_sequence[(start_res-1):(start_res + k - 1)]
        running_df = pd.DataFrame(np.repeat(fasta_slice,traj.n_frames),columns=['fasta_sequence'])
        running_df['start_res_zeroindex'] = np.repeat(start_res-1,traj.n_frames)
        running_df['end_res_zeroindex'] = np.repeat(start_res + k - 2,traj.n_frames)
        running_df['full_protein_fasta'] = np.repeat(fasta_sequence,traj.n_frames)

        running_df['full_protein_rgyr'] = np.repeat(complete_protein_rgyr,traj.n_frames)
        running_df['full_protein_nu_recompute'] = np.repeat(complete_protein_nu,traj.n_frames)
        running_df['full_protein_nu_recompute_err'] = np.repeat(complete_protein_nu_err,traj.n_frames)
        running_df['full_protein_ratio'] = np.repeat(complete_protein_inst_ratio,traj.n_frames)
        running_df['full_protein_RSA'] = np.repeat(complete_protein_RSA,traj.n_frames)
        running_df['full_protein_rg_rg_theta_mean'] = np.repeat(complete_protein_rg_rg_theta_mean,traj.n_frames)
        
        running_df['seq_name'] = np.repeat(seq_name,traj.n_frames)
        # Create a trajectory slice for the selected subsequence
        subsequence_traj = traj.atom_slice(subsequence_indices)
        
        # Calculate the radius of gyration for the subsequence over all remaining frames
        rgyr = md.compute_rg(subsequence_traj)
        running_df['Rg/nm'] = rgyr

        #calculate nu
        subsequence_nu = calculate_nu_KLL_from_seq_name(seq_name, start_res, start_res + k - 1)[0]
        subsequence_nu_err = calculate_nu_KLL_from_seq_name(seq_name, start_res, start_res + k - 1)[1]
        running_df['nu_recompute'] = np.repeat(subsequence_nu,traj.n_frames)
        running_df['nu_recompute_err'] = np.repeat(subsequence_nu_err,traj.n_frames)
        
        # Select indices for the first and last bead in the subsequence for end-to-end distance calculation
        first_bead_index = traj.topology.select(f"residue {start_res-1}")[0]
        last_bead_index = traj.topology.select(f"residue {start_res + k - 2}")[0]
    
        # Calculate end-to-end distances for the subsequence over all remaining frames
        end_to_end_distances = md.compute_distances(traj, [[first_bead_index, last_bead_index]])
        end_to_end_distances = end_to_end_distances.flatten()
        running_df['ete'] = end_to_end_distances
        running_df['inst_ratio'] = (running_df['ete'].values**2)/(running_df['Rg/nm'].values**2)
        
        t_df_moments = pd.DataFrame(md.principal_moments(subsequence_traj),columns=['R3','R2','R1']).copy()
        t_df_moments['asphericity']=t_df_moments.R1.values-(0.5*(t_df_moments.R2.values+t_df_moments.R3.values))
        t_df_moments['acylindricity']=t_df_moments.R2.values-t_df_moments.R3.values
        t_df_moments['RSA']=(t_df_moments.asphericity.values**2+(0.75*t_df_moments.acylindricity.values**2))/(t_df_moments.R1.values+t_df_moments.R2.values+t_df_moments.R3.values)**2
        running_df['RSA'] = t_df_moments['RSA'].values

        #AFRC
        afrc_init = afrc.AnalyticalFRC(fasta_slice)
        rg_theta_mean = afrc_init.get_mean_radius_of_gyration()
        running_df['AFRC_mean_rg_theta'] = np.repeat(rg_theta_mean,traj.n_frames)
        running_df['rg_rg_theta_mean'] = (10*running_df['Rg/nm'].values)/running_df['AFRC_mean_rg_theta'].values
        
        if j == 0:
            master_df = running_df.copy()
        elif j>0:
            master_df = pd.concat([master_df,running_df],axis=0).copy()
        del running_df, t_df_moments
        print(f"residue {start_res-1} to {start_res + k - 2} done")
        j+=1
    print('ALL DONE')    
    return master_df

In [66]:

test_verify = orig_compute_kmer_data_from_seq_name('A0A087WTH5_1_132')

residue 0 to 29 done
residue 1 to 30 done
residue 2 to 31 done
residue 3 to 32 done
residue 4 to 33 done
residue 5 to 34 done
residue 6 to 35 done
residue 7 to 36 done
residue 8 to 37 done
residue 9 to 38 done
residue 10 to 39 done
residue 11 to 40 done
residue 12 to 41 done
residue 13 to 42 done
residue 14 to 43 done
residue 15 to 44 done
residue 16 to 45 done
residue 17 to 46 done
residue 18 to 47 done
residue 19 to 48 done
residue 20 to 49 done
residue 21 to 50 done
residue 22 to 51 done
residue 23 to 52 done
residue 24 to 53 done
residue 25 to 54 done
residue 26 to 55 done
residue 27 to 56 done
residue 28 to 57 done
residue 29 to 58 done
residue 30 to 59 done
residue 31 to 60 done
residue 32 to 61 done
residue 33 to 62 done
residue 34 to 63 done
residue 35 to 64 done
residue 36 to 65 done
residue 37 to 66 done
residue 38 to 67 done
residue 39 to 68 done
residue 40 to 69 done
residue 41 to 70 done
residue 42 to 71 done
residue 43 to 72 done
residue 44 to 73 done
residue 45 to 74 don

In [108]:
test_verify['mid_residue'] = (test_verify['start_res_zeroindex'] + test_verify['end_res_zeroindex']) //2
test_verify[test_verify.mid_residue==57]['rg_rg_theta_mean'].mean()

1.015958833716071

In [80]:
test

,seq_name,window_size,min_mean_Rg,mid_res_min_mean_Rg,max_mean_Rg,mid_res_max_mean_Rg,stdev_mean_Rg,min_mean_Rs,mid_res_min_mean_Rs,max_mean_Rs,...,min_mean_rg_rg_theta,mid_res_min_mean_rg_rg_theta,max_mean_rg_rg_theta,mid_res_max_mean_rg_rg_theta,stdev_mean_rg_rg_theta,min_mean_nu,mid_res_min_mean_nu,max_mean_nu,mid_res_max_mean_nu,stdev_mean_nu
0,A0A087WTH5_1_132,30,1.386801,35,1.498327,116,0.033564,5.529051,51,6.313742,...,1.015959,57,1.093254,116,0.021404,0.518506,50,0.587438,113,0.019152


## tests complete

In [20]:
os.listdir('subchain_repository_hdf5/')

['D6RBM5_1_74.h5',
 'Q6IE37_888_1185.h5',
 'Q8TC71_76_123.h5',
 'Q8N884_1_165.h5',
 'Q9BUX1_1_30.h5',
 'P42127_1_92.h5',
 'Q92804_317_592.h5',
 'P31689_66_102.h5',
 'Q15653_303_356.h5',
 'P21754_325_424.h5',
 'P19957_1_70.h5',
 'Q86TB3_2128_2170.h5',
 'P78552_1_34.h5',
 'Q96KR4_607_655.h5',
 'Q32NB8_1_80.h5',
 'P00747_450_487.h5',
 'O00512_1_179.h5',
 'O75113_188_344.h5',
 'Q8TA86_176_221.h5',
 'O14633_1_110.h5',
 'Q15031_1_45.h5',
 'Q8N7X0_1172_1364.h5',
 'O60507_25_56.h5',
 'P07900_694_732.h5',
 'Q16891_1_208.h5',
 'Q9Y3B9_210_282.h5',
 'Q13258_324_359.h5',
 'Q5VXU9_1_589.h5',
 'Q9NY12_1_67.h5',
 'P35218_1_58.h5',
 'Q15758_489_541.h5',
 'Q14938_179_502.h5',
 'Q8WWV6_485_532.h5',
 'Q13562_1_85.h5',
 'O75864_464_691.h5',
 'Q6PIJ6_1_31.h5',
 'P04632_1_92.h5',
 'A6NC57_530_561.h5',
 'Q5T5N4_1_200.h5',
 'A6NHR9_438_469.h5',
 'Q96JS3_1_47.h5',
 'Q9NZS9_122_160.h5',
 'P50851_933_1306.h5',
 'P61925_1_76.h5',
 'A0AUZ9_413_987.h5',
 'Q8NCU4_270_306.h5',
 'O95390_107_143.h5',
 'Q8IZY2_1409_1449

In [22]:
def explore_hdf5_file(file_path):
    with h5py.File(file_path, 'r') as file:
        print("File structure:")
        file.visit(print)  # This will print all groups and datasets in the file

In [61]:


# Check the structure of your HDF5 file
explore_hdf5_file('./test_folder/test.h5')

File structure:
A0A024RBG1_145_181
A0A024RBG1_145_181/fasta_sequence_whole_protein
A0A024RBG1_145_181/full_protein_RSA
A0A024RBG1_145_181/full_protein_nu_recompute
A0A024RBG1_145_181/full_protein_nu_recompute_err
A0A024RBG1_145_181/full_protein_ratio
A0A024RBG1_145_181/full_protein_rg_rg_theta_mean
A0A024RBG1_145_181/full_protein_rgyr
A0A024RBG1_145_181/mer_1_30
A0A024RBG1_145_181/mer_1_30/RSA
A0A024RBG1_145_181/mer_1_30/Rg
A0A024RBG1_145_181/mer_1_30/Rg/nm
A0A024RBG1_145_181/mer_1_30/ete
A0A024RBG1_145_181/mer_1_30/fasta_subsequence
A0A024RBG1_145_181/mer_1_30/inst_ratio
A0A024RBG1_145_181/mer_1_30/nu_recompute
A0A024RBG1_145_181/mer_1_30/nu_recompute_err
A0A024RBG1_145_181/mer_1_30/rg_rg_theta_mean
A0A024RBG1_145_181/mer_1_30/selection_string
A0A024RBG1_145_181/mer_2_31
A0A024RBG1_145_181/mer_2_31/RSA
A0A024RBG1_145_181/mer_2_31/Rg
A0A024RBG1_145_181/mer_2_31/Rg/nm
A0A024RBG1_145_181/mer_2_31/ete
A0A024RBG1_145_181/mer_2_31/fasta_subsequence
A0A024RBG1_145_181/mer_2_31/inst_ratio
A0A

In [62]:
def load_dataset_to_dataframe(file_path, protein_name, dataset_name):
    with h5py.File(file_path, 'r') as file:
        full_path = f"{protein_name}/{dataset_name}"
        if full_path in file:
            data = file[full_path][...]
            # Check if the data is a scalar (has no dimensions)
            if data.shape == ():
                # Convert scalar to a 1D array with one element
                data = np.array([data])
                return pd.DataFrame(data, columns=[dataset_name])
            else:
                # Handle 1D array data
                return pd.DataFrame(data, columns=[dataset_name])
        else:
            raise ValueError("Dataset path does not exist in the file.")


In [72]:
df_fasta = load_dataset_to_dataframe('./test_folder/test.h5', 'A0A024RBG1_145_181', 'seq_name')
df_fasta

,seq_name
0,b'A0A024RBG1_145_181'


## test actual data

In [24]:
# Check the structure of your HDF5 file
explore_hdf5_file('./subchain_repository_hdf5/P35368_370_520.h5')

File structure:
P35368_370_520
P35368_370_520/fasta_sequence_whole_protein
P35368_370_520/full_protein_RSA
P35368_370_520/full_protein_nu_recompute
P35368_370_520/full_protein_nu_recompute_err
P35368_370_520/full_protein_ratio
P35368_370_520/full_protein_rg_rg_theta_mean
P35368_370_520/full_protein_rgyr
P35368_370_520/mer_100_129
P35368_370_520/mer_100_129/RSA
P35368_370_520/mer_100_129/Rg
P35368_370_520/mer_100_129/Rg/nm
P35368_370_520/mer_100_129/ete
P35368_370_520/mer_100_129/fasta_subsequence
P35368_370_520/mer_100_129/inst_ratio
P35368_370_520/mer_100_129/nu_recompute
P35368_370_520/mer_100_129/nu_recompute_err
P35368_370_520/mer_100_129/rg_rg_theta_mean
P35368_370_520/mer_100_129/selection_string
P35368_370_520/mer_101_130
P35368_370_520/mer_101_130/RSA
P35368_370_520/mer_101_130/Rg
P35368_370_520/mer_101_130/Rg/nm
P35368_370_520/mer_101_130/ete
P35368_370_520/mer_101_130/fasta_subsequence
P35368_370_520/mer_101_130/inst_ratio
P35368_370_520/mer_101_130/nu_recompute
P35368_370_52

In [25]:
subchain = pd.read_csv('TESTING_parallelization_store_all_subchains_Jesse_moving_window_HDF5.csv')
subchain

,Unnamed: 0,seq_name,window_size,min_mean_Rg,mid_res_min_mean_Rg,max_mean_Rg,mid_res_max_mean_Rg,stdev_mean_Rg,min_mean_Rs,mid_res_min_mean_Rs,...,min_mean_rg_rg_theta,mid_res_min_mean_rg_rg_theta,max_mean_rg_rg_theta,mid_res_max_mean_rg_rg_theta,stdev_mean_rg_rg_theta,min_mean_nu,mid_res_min_mean_nu,max_mean_nu,mid_res_max_mean_nu,stdev_mean_nu
0,0,A0A024RBG1_145_181,12,0.787629,6,0.828673,19,0.015109,6.321465,28,...,0.913884,8,0.954280,18,0.014482,0.604528,13,0.655475,18,0.013469
1,0,A0A075B6T7_1_32,11,0.748416,24,0.789972,12,0.012647,6.131754,19,...,0.908635,24,0.962646,12,0.014967,0.575042,18,0.645993,7,0.019511
2,0,A0A087WTH1_1_67,30,1.406829,19,1.472289,51,0.018825,5.698813,14,...,1.022500,19,1.073180,51,0.013978,0.524312,19,0.592436,47,0.024612
3,0,A0A087WTH5_1_132,30,1.386801,35,1.498327,116,0.033564,5.529051,51,...,1.015959,57,1.093254,116,0.021404,0.518506,50,0.587438,113,0.019152
4,0,A0A087WV53_1_46,15,0.930813,15,0.985087,7,0.016192,5.840974,31,...,0.955543,31,1.010013,22,0.017341,0.553039,32,0.645676,20,0.029551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,0,A2IDD5_317_438,30,1.379039,91,1.513469,21,0.037775,5.609603,100,...,1.009741,91,1.104647,22,0.026007,0.529038,100,0.585359,20,0.012630
277,0,A2IDD5_102_166,30,1.422826,14,1.496717,46,0.016616,6.014925,49,...,1.040120,14,1.097931,47,0.013850,0.564297,14,0.590320,20,0.007207
278,0,A2PYH4_1091_1435,30,1.397212,201,1.651645,176,0.045287,5.477315,197,...,1.026226,202,1.207664,176,0.033766,0.514635,197,0.655160,174,0.026100
279,0,A2PYH4_1_266,30,1.406721,176,1.591095,41,0.044717,5.791986,49,...,1.024078,225,1.159552,41,0.032804,0.540005,176,0.646296,39,0.021089
